This Notebook evaluates test cases for the 1D Shock-Acoustic-Wave interaction problem as presented presented in the phd thesis "Implicit Discontinuous Galerkin Shock Tracking methods for Compressible Flows with Shocks" (Vandergrift 2024).

In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\XESTSF.dll"
using System;
using BoSSS.Foundation;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution;
using BoSSS.Solution.Tecplot;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Solution.Utils;
using ilPSP.LinSolvers;
using BoSSS.Solution.NSECommon;
using ilPSP.Connectors.Matlab;
using ilPSP;
using BoSSS.Foundation.Grid.Classic;
using ilPSP.Utils;
using BoSSS.Foundation.Grid.RefElements;
using System.Collections.Generic;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.XDG;
using BoSSS.Solution.XdgTimestepping;
using BoSSS.Solution.AdvancedSolvers;
using System.Linq;
using BoSSS.Foundation.Grid.Aggregation;
using BoSSS.Platform;
using MPI.Wrappers;
using System.Diagnostics;
using System.IO;
using System.Collections;
using BoSSS.Foundation.Quadrature;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Gnuplot;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using ApplicationWithIDT;
using XESTSF;
using XDGShock;
Init();


In [2]:
var ass= new MatrixAssembler(1);
BoSSSshell.WorkflowMgm.Init("XESTSF_ShockAcousticInteraction1D");
BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();
BoSSSshell.WorkflowMgm.DefaultDatabase


{ Session Count = 12; Grid Count = 12; Path = \\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D }

In [3]:
string dbPath = BoSSSshell.WorkflowMgm.DefaultDatabase.Path; 
dbPath

\\dc3\userspace\sebastian\cluster\XESTSF_ShockAcousticInteraction1D

function to plot line outs of the space time solution

In [4]:
(double[] xVals, double[]yVals) GetPlot(double xMin,double xMax, double t, int nRef,string field,ISessionInfo si){
//double shockPosition=0.5;
var p1= new double[] {xMin,t};
var p2=new double[] {xMax,t};

//timesteps to compare
var tiEnd=si.Timesteps.Last();

// # use p_per from BoSSS 
var pEnd =tiEnd.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

In [5]:
var db=OpenOrCreateDatabase(dbPath);
var ses=db.Sessions;
ses

#0: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:48:38 PM	50cba6ca...
#1: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:48:01 PM	3fb780f8...
#2: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	4/10/2024 2:46:46 PM	9bcce582...
#3: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:46:07 PM	012246ad...
#4: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:45:27 PM	90107f8d...
#5: XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8	4/10/2024 2:47:24 PM	8c0c4d79...
#6: XESTSF_AcousticWave1D	AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5

In [6]:
var si=db.Sessions.Pick(0);
var ti=si.Timesteps.Last();
//si.Timesteps
si

XESTSF_AcousticWave1D	AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8	4/10/2024 2:48:38 PM	50cba6ca...

# Plotting

define general params for plotting (must substract some eps because, otherwise some points are not located in the space-time grid)

In [7]:
var eps=0.000001;
double xMin=0 +eps;double xMax=3.0-eps; int xRef=100; 
double tMin =0 + eps;int tRef=100;
int nRows=2;

obtain tMax from session name

In [8]:
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;

## Pressure Pertubations

In [ ]:
using System.Text.RegularExpressions;
foreach(var si in ses){
    string pattern = @"tMax(\d+(\.\d+)?)";
    Match match = Regex.Match(si.Name, pattern);
    double tMax = 1.5;
    try{
        tMax = double.Parse(match.Groups[1].Value)-eps;
    }catch{
    }
    Console.WriteLine($"doing:{si.Name}; tMax:{tMax}");
    string field="p_per";
    string csvFileName = si.Name + "_p_per.csv";
    Plot2Ddata[,] multiplotsPressure = new Plot2Ddata[(tRef+1)/nRows,nRows];
    using (StreamWriter writer = new StreamWriter(csvFileName))
            {
            for(int Ti=0;Ti< tRef;Ti++){
                var plot= new Plot2Ddata();
                plot.Xlabel="x";
                plot.Ylabel="p'";
                double t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
                //Console.WriteLine(t);
                (double[] xVals,double[] yVals)=GetPlot(xMin,xMax,t,xRef,field, si);
                plot.AddDataGroup("t="+Math.Round(t,2),xVals,yVals,fmt);
                multiplotsPressure[Ti/nRows,Ti %nRows]=plot;
                if (Ti == 0)
                {
                    // Write header row
                    writer.WriteLine($"x/t,{string.Join(",", xVals)}");
                }
                // Write data row
                writer.WriteLine($"{Math.Round(t, 2)},{string.Join(",", yVals)}");
                        
            }
    };
}



doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999


In [ ]:
foreach(var si in ses){
    IDTTimeStepInfo lts = (IDTTimeStepInfo)((TimestepProxy)si.Timesteps.Last()).GetInternal();
    lts.EnResHistory.SaveToTextFile(si.Name + "_EnResNorms.txt");
    lts.ResHistory.SaveToTextFile(si.Name + "_ResNorms.txt");
}
//Array.ForEach(multiplotsPressure.Cast<Plot2Ddata>().ToArray(), 
//    plot => { plot.YrangeMin = -1.5 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });
// multiplotsPressure[1,1].YrangeMax = 5 * 1e-7; multiplotsPressure[1,1].YrangeMin = -6 * 1e-7;
// multiplotsPressure[2,0].YrangeMax = 5 * 1e-7; multiplotsPressure[2,0].YrangeMin = -6 * 1e-7;
// multiplotsPressure[2,1].YrangeMax = 5 * 1e-7; multiplotsPressure[2,1].YrangeMin = -6 * 1e-7;
//multiplotsPressure.ToGnuplot().PlotSVG(xRes:900,yRes:600)

## Density Pertubations

In [10]:
using System.Text.RegularExpressions;
foreach(var si in ses){
        string pattern = @"tMax(\d+)";
    Match match = Regex.Match(si.Name, pattern);
    double tMax = 1.5;
    try{
        tMax = double.Parse(match.Groups[1].Value)-eps;
    }catch{
    }
    Console.WriteLine($"doing:{si.Name}; tMax:{tMax}");
    string field="rho_per";
    // Format
    var fmt= new PlotFormat();
    fmt.WithPointSize(0.0);
    fmt.LineColor=LineColors.Red;

    Plot2Ddata[,] multiplotsDensity = new Plot2Ddata[(tRef+1)/nRows,nRows];
    string csvFileName = si.Name + "_rho_per.csv";
    using (StreamWriter writer = new StreamWriter(csvFileName)){
        for(int Ti=0;Ti< tRef;Ti++){
            var plot= new Plot2Ddata();
            plot.Xlabel="x";
            plot.Ylabel="rho'";
            double t=((double) tRef-1-Ti)/((double) tRef-1)*tMin+((double) Ti)/(tRef-1)*tMax;
            //Console.WriteLine(t);
            (double[] xVals,double[] yVals)=GetPlot(xMin,xMax,t,xRef,field, si);
            plot.AddDataGroup("t="+Math.Round(t,2),xVals,yVals,fmt);
            multiplotsDensity[Ti/nRows,Ti %nRows]=plot;
            if (Ti == 0)
            {
                // Write header row
                writer.WriteLine($"x/t,{string.Join(",", xVals)}");
            }
            // Write data row
            writer.WriteLine($"{Math.Round(t, 2)},{string.Join(",", yVals)}");
                    
        }
        
    };                                              
}

doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells61_tCells61_tMax4_1sinus_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax4_bump_Mach1.5_sP0.5_wP0.9_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells61_tCells61_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:0.999999
doing:AW_p3_xCells91_tCells91_tMax4_1sinus_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax4_bump_Mach1.5_sP1.5_wP0.3_ampneg1E-05_amppos0_wLTrue_wSTrue_wL0.8; tMax:3.999999
doing:AW_p3_xCells91_tCells91_tMax1.5_1sinus_Mach1.5_sP1.5_wP0_a

In [ ]:
// Array.ForEach(multiplotsDensity.Cast<Plot2Ddata>().ToArray(), 
//     plot => { plot.YrangeMin = -1 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });
// // multiplotsDensity[1,1].YrangeMax = 5 * 1e-7; multiplotsDensity[1,1].YrangeMin = -6 * 1e-7;
// //multiplotsDensity[2,0].YrangeMax = 5 * 1e-7; multiplotsDensity[2,0].YrangeMin = -6 * 1e-7;
// //multiplotsDensity[2,1].YrangeMax = 5 * 1e-7; multiplotsDensity[2,1].YrangeMin = -6 * 1e-7;
// multiplotsDensity.ToGnuplot().PlotSVG(xRes:1000,yRes:600)

In [ ]:
(double[] xVals, double[]yVals) GetPlotLevelSet(double tMin,double tMax, int nRef,ISessionInfo si){
//double shockPosition=0.5;
var p1= new double[] {0.0,tMin};
var p2=new double[] {0.0,tMax};

//timesteps to compare
var tiEnd=si.Timesteps.Last();
// # use p_per from BoSSS 
var pEnd =tiEnd.GetField("LevelSet");
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(xMin,xMax,nRef);
return(xVals,yVals);
}

In [ ]:
// Format
var fmt= new PlotFormat();
fmt.WithPointSize(0.0);
fmt.LineColor=LineColors.Red;
string csvFileName = "XESTSFLevelSetPertubations.csv";
var plotLevelSet= new Plot2Ddata();
using (StreamWriter writer = new StreamWriter(csvFileName)){
    
    //Console.WriteLine(t);
    (double[] xVals,double[] yVals)=GetPlotLevelSet(tMin,tMax,1000, si);
    
    plotLevelSet.AddDataGroup("LevelSet",xVals,yVals,fmt);
        // Write header row
    writer.WriteLine($"t,{string.Join(",", xVals)}");
    
    // Write data row
    writer.WriteLine($"phi(t),{string.Join(",", yVals)}");
            
}

plotLevelSet.Ylabel="\varphi(0,t)";
plotLevelSet.ToGnuplot().PlotSVG(xRes:800,yRes:800)  


Error: (10,58): error CS0103: The name 'tMax' does not exist in the current context

plot along ray

In [ ]:
using BoSSS.Solution.CompressibleFlowCommon;
double densityL=1, pressureL=1,MachL=1.5,gamma=1.4;
double cL = Math.Sqrt(gamma * pressureL / densityL);
double velocityL = MachL * cL;
(double densityR, double velocityR, double pressureR, double cR, double MachR)
    = CompressibleHelperFunc.ComputeNormalShockWaveRelations(densityL, velocityL, pressureL, MachL, gamma);
(double[] xVals, double[]yVals) GetPlotAlongRay(double xMax, int nRef,string field,ISessionInfo si, double xa, double xs){
//double shockPosition=0.5;
var inc_speed= xa<xs? (velocityL + cL): (velocityR - cR);
var tstar= (xs-xa)/inc_speed;
Func<double,double> X_s = t => (velocityR + cR)*t+ (xs-tstar*(velocityR + cR));
double tEnd= (xMax - (xs-tstar*(velocityR + cR)))/(velocityR + cR);
var eps=5e-2;
var p1= new double[] {X_s(tstar+eps),tstar+eps};
var p2=new double[] {xMax,tEnd};

//timesteps to compare
var tiEnd=si.Timesteps.Last();

// # use p_per from BoSSS 
var pEnd =tiEnd.GetField(field);
double[] yVals=pEnd.EvaluateAlongLine(p1,p2,nRef);
double[] xVals= GenericBlas.Linspace(tstar,tEnd,nRef);
return(xVals,yVals);
}

Plot2Ddata[,] multiplotsPressure = new Plot2Ddata[1,1];
foreach(var si in ses.Where(si => si.Name.Contains("_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8"))){
    //catch tmax value
    string pattern = @"tMax(\d+(\.\d+)?)";
    Match match = Regex.Match(si.Name, pattern);
    double tMax = 1.5;
    try{
        tMax = double.Parse(match.Groups[1].Value)-eps;
    }catch{
    }

    // catch shock position
    pattern = @"sP(\d+(\.\d+)?)";
    Match match2 = Regex.Match(si.Name, pattern);
    double xs = 1.5;
    try{
        xs = double.Parse(match2.Groups[1].Value);
    }catch{
    }

    // catch wave positions
    pattern = @"wP(\d+(\.\d+)?)";
    match = Regex.Match(si.Name, pattern);
    double wP = 0.0;
    try{
        wP = double.Parse(match.Groups[1].Value);
    }catch{
    }

    // catch wave length and convert to position of amplitude
    pattern = @"wL(\d+(\.\d+)?)";
    match = Regex.Match(si.Name, pattern);
    double xa = 1.5;
    try{
        if(si.Name.Contains("1sinus")){
            xa = wP+double.Parse(match.Groups[1].Value)/4;
        }else{
            xa = wP+double.Parse(match.Groups[1].Value)/2;
        }
        
    }catch{
    }




    Console.WriteLine($"doing:{si.Name}; tMax:{tMax}");
    Console.WriteLine($"Catched: wP{wP}, xa{xa},xs{xs}");
    string field="p_per";
    string csvFileName = si.Name + "_p_per_Ray.csv";
    using (StreamWriter writer = new StreamWriter(csvFileName))
        {
            var plot= new Plot2Ddata();
            plot.Xlabel="x";
            plot.Ylabel="p'";
            //Console.WriteLine(t);
            (double[] xVals,double[] yVals)=GetPlotAlongRay(xMax,xRef,field, si,xa,xs);
            Console.WriteLine($"AverageAlongRay: {yVals.Average()}");
            plot.AddDataGroup("Ray",xVals,yVals,fmt);
            multiplotsPressure[0,0]=plot;
            // Write header row
            writer.WriteLine($"x/t,{string.Join(",", xVals)}");
            
            // Write data row
            writer.WriteLine($"Ray,{string.Join(",", yVals)}");
                    
        }
};

multiplotsPressure.ToGnuplot().PlotSVG(xRes:900,yRes:600)
//    plot => { plot.YrangeMin = -1.5 * 1e-5; plot.YrangeMax = 1.5 * 1e-5; });A


doing:AW_p3_xCells61_tCells61_tMax1.5_bump_Mach1.5_sP1.5_wP0_ampneg0_amppos1E-05_wLTrue_wSTrue_wL0.8; tMax:1.499999
Catched: wP0, xa0.4,xs1.5
AverageAlongRay: 2.1574504113916715E-05
Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
Note: In a Jupyter Worksheet, you must NOT have a trailing semicolon in order to see the plot on screen; otherwise, the output migth be surpressed.!


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00002154 
 
 
 
 
 0.00002155 
 
 
 
 
 0.00002156 
 
 
 
 
 0.00002157 
 
 
 
 
 0.00002158 
 
 
 
 
 0.00002159 
 
 
 
 
 0.00002160 
 
 
 
 
 0.00002161 
 
 
 
 
 0.00002162 
 
 
 
 
 0.3 
 
 
 
 
 0.4 
 
 
 
 
 0.5 
 
 
 
 
 0.6 
 
 
 
 
 0.7 
 
 
 
 
 0.8 
 
 
 
 
 0.9 
 
 
 
 
 1 
 
 
 
 
 1.1 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 p' 
 
 
 
 
 x 
 
 
 
 
 Ray 
 
 
 Ray 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M735.0,63.1 L788.4,63.1 M194.4,505.4 L200.2,510.2 L206.1,487.7 L211.9,480.0 L217.7,489.7 L223.5,474.4
 L229.3,454.8 L235.2,444.9 L241.0,463.9 L246.8,437.5 L252.6,423.9 L258.4,416.3 L264.3,430.1 L270.1,416.3
 L275.9,410.2 L281.7,406.2 L287.5,422.1 L293.3,420.3 L299.2,421.0 L305.0,437.0 L310.8,438.8 L316.6,441.9
 L322.4,463.1 L328.3,467.3 L334.1,466.4 L339.9,458.3 L345.7,472.0 L351.5,463.4 L357.3,483.7 L363.2,480.8
 L369.0,440.7 L374.8,423.7 L380.6,474.6 L386.4,424.6 L392.3,387.5 L398.1,370.4 L403.9,417.1 L409.7,363.5
 L415.5,339.7 L421.4,330.0 L427.2,304.0 L433.0,323.2 L438.8,316.5 L444.6,298.8 L450.4,288.5 L456.3,313.7
 L462.1,319.6 L467.9,318.4 L473.7,320.5 L479.5,326.9 L485.4,345.7 L491.2,342.7 L497.0,343.1 L502.8,342.2
 L508.6,353.5 L514.4,338.9 L520.3,332.5 L526.1,370.0 L531.9,325.1 L537.7,305.4 L543.5,294.4 L549.4,314.8
 L555.2,276.1 L561.0,260.2 L566.8,292.0 L572.6,252.8 L578.5,230.6 L584.3,224.9 L590.1,184.4 L595.9,208.8
 L601.7,206.5 L607.5,178.8 L613.4,178.1 L619.2,194.9 L625.0,207.8 L630.8,206.7 L636.6,210.2 L642.5,206.8
 L648.3,238.7 L654.1,234.5 L659.9,233.7 L665.7,226.5 L671.6,250.9 L677.4,234.8 L683.2,228.5 L689.0,271.1
 L694.8,225.9 L700.6,206.4 L706.5,198.3 L712.3,216.0 L718.1,179.2 L723.9,165.5 L729.7,154.7 L735.6,152.1
 L741.4,134.3 L747.2,132.8 L753.0,86.2 L758.8,105.4 L764.6,110.9 L770.5,122.9 '/>